<a href="https://colab.research.google.com/github/mahdimalverdi/ACM/blob/master/Exam1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

In [5]:
import findspark
findspark.init()

In [6]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DoubleType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col,array_contains
spark = SparkSession.builder.master("local[1]").appName("hw6").getOrCreate()

In [25]:
!unzip files.zip


Archive:  files.zip
   creating: datasets/
  inflating: datasets/covid19.rar    
  inflating: datasets/dataset2.xlsx  
   creating: covid19/
   creating: covid19/data/
  inflating: covid19/data/data.csv   
  inflating: covid19/data/labels.csv  


In [7]:
train_dataset = spark.read.option("maxColumns", 50_000).csv("covid19/data/data.csv",inferSchema=False, header =False)
train_dataset.printSchema()


root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (nullable = tru

In [16]:
def getHeaders(n):
  return n.name

headers = list(map(getHeaders,train_dataset.schema.fields))

In [ ]:
for header in headers:
  train_dataset = train_dataset.withColumn(header, DoubleType)

In [19]:
train_dataset.head(1)

[Row(_c0='1', _c1='0.5', _c2='0.5', _c3='1', _c4='1', _c5='1', _c6='0.75', _c7='0.75', _c8='0.5', _c9='0.5', _c10='0.5', _c11='1', _c12='0.5', _c13='1', _c14='0.25', _c15='0.25', _c16='0.5', _c17='0.5', _c18='0.25', _c19='0.25', _c20='0.25', _c21='1', _c22='0.75', _c23='0.75', _c24='0.5', _c25='1', _c26='1', _c27='0.25', _c28='1', _c29='1', _c30='1', _c31='0.25', _c32='0.25', _c33='1', _c34='1', _c35='0.25', _c36='0.25', _c37='1', _c38='1', _c39='0.25', _c40='0.5', _c41='0.5', _c42='0.5', _c43='0.25', _c44='0.75', _c45='1', _c46='0.5', _c47='0.25', _c48='0.5', _c49='0.25', _c50='0.5', _c51='0.5', _c52='0.75', _c53='0.5', _c54='1', _c55='0.75', _c56='1', _c57='0.5', _c58='0.25', _c59='0.5', _c60='0.75', _c61='0.5', _c62='0.5', _c63='0.25', _c64='0.5', _c65='0.25', _c66='0.5', _c67='1', _c68='1', _c69='1', _c70='0.25', _c71='0.75', _c72='1', _c73='1', _c74='0.25', _c75='0.5', _c76='0.5', _c77='0.5', _c78='1', _c79='1', _c80='1', _c81='1', _c82='0.5', _c83='0.25', _c84='0.5', _c85='0.75',

In [17]:
from pyspark.ml.feature import VectorAssembler
import pyspark.sql.functions as F

assembler = VectorAssembler(inputCols=headers,outputCol = 'Attributes')
mapping= {
      'setosa': 1.0,
      'versicolor': 2.0,
      'virginica': 3.0
  }

  

output = assembler.transform(train_dataset)
finalized_data = output.select("Attributes","number_class")
finalized_data.show()

IllegalArgumentException: ignored